In [12]:
import fitz  # PyMuPDF

doc = fitz.open(r"E:\SEMESTER 1\PF~ADIL SHEHZAD\BOOKS\C++ How to Program, 10th edition.pdf")




In [17]:
from IPython.core.interactiveshell import InteractiveShell

# Show all output, not just last expression
InteractiveShell.ast_node_interactivity = "all"

# Increase max output length
from IPython.display import HTML
import sys
sys.stdout.write = lambda x: HTML(x)

In [20]:
for page_num in range(10):#len(doc)):
    page = doc[page_num]
    print(page.get_text())    # Get text with formatting
    text = page.get_text()
    
    # Get text with font info (bold, size, etc.)
    blocks = page.get_text("dict")["blocks"]
    
    # Get images
    images = page.get_images()
    
    print(f"Page {page_num + 1}:")
    print(f"  Text blocks: {len(blocks)}")
    print(f"  Images: {len(images)}")

dumperina

Page 1:
  Text blocks: 2
  Images: 1

Page 2:
  Text blocks: 0
  Images: 1
This page intentionally left blank

Page 3:
  Text blocks: 1
  Images: 0
Paul Deitel
Deitel & Associates, Inc.
Harvey Deitel
Deitel & Associates, Inc.

Page 4:
  Text blocks: 2
  Images: 1
Vice President, Editorial Director: Marcia Horton
Acquisitions Editor: Tracy Johnson
Editorial Assistant: Kristy Alaura
Acquisitions Editor, Global Editions: Sourabh Maheshwari
VP of Marketing: Christy Lesko
Director of Field Marketing: Tim Galligan
Product Marketing Manager: Bram Van Kempen
Field Marketing Manager: Demetrius Hall
Marketing Assistant: Jon Bryant
Director of Product Management: Erin Gregg
Team Lead, Program and Project Management: Scott Disanno
Program Manager: Carole Snyder
Project Manager: Robert Engelhardt
Project Editor, Global Editions: K.K. Neelakantan
Senior Manufacturing Controller, Global Editions: Trudy Kimber
Senior Specialist, Program Planning and Support: Maura Zaldivar-Garcia
Media Prod

In [22]:
import fitz  # PyMuPDF
from PIL import Image
import io



# Extract images from a specific page
page_num = 0  # First page
page = doc[page_num]

# Get all images on the page
images = page.get_images()

print(f"Found {len(images)} images on page {page_num + 1}")

# Extract each image
for img_index, img in enumerate(images):
    # Get image reference
    xref = img[0]
    
    # Extract image bytes
    base_image = doc.extract_image(xref)
    image_bytes = base_image["image"]
    image_ext = base_image["ext"]  # png, jpeg, etc.
    
    # Save image to file
    with open(f"image_page{page_num + 1}_{img_index + 1}.{image_ext}", "wb") as f:
        f.write(image_bytes)
    
    print(f"✓ Saved: image_page{page_num + 1}_{img_index + 1}.{image_ext}")

Found 1 images on page 1


597371

✓ Saved: image_page1_1.jpeg


In [3]:
import re
from typing import List, Dict
import pdfplumber
import tiktoken

def chunk_pdf_for_rag(
    pdf_path: str,
    max_tokens: int = 500,
    overlap_tokens: int = 75,
    encoding_name: str = "cl100k_base"
) -> List[Dict]:
    """
    Token-accurate, sentence-based chunking for RAG.
    Returns chunks with page range and chapter metadata.
    """

    enc = tiktoken.get_encoding(encoding_name)

    def count_tokens(text: str) -> int:
        return len(enc.encode(text))

    def detect_chapter(text: str):
        match = re.search(r'\b(chapter|chap)\s+(\d+)\b', text, re.IGNORECASE)
        return int(match.group(2)) if match else None

    def split_into_sentences(text: str) -> List[str]:
        return re.split(r'(?<=[.!?])\s+', text.strip())

    chunks = []
    current_chunk = ""
    current_tokens = 0
    current_chapter = None
    chunk_start_page = None

    i = 0
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            raw_text = page.extract_text() or ""
            sentences = split_into_sentences(raw_text)

            for sentence in sentences:
                sentence = sentence.strip()
                if not sentence:
                    continue

                # Detect chapter headings
                chapter = detect_chapter(sentence)
                if chapter is not None and chapter != current_chapter:
                    # Flush chunk on chapter boundary
                    if current_chunk.strip():
                        chunks.append({
                            "text": current_chunk.strip(),
                            "page_start": chunk_start_page,
                            "page_end": page_num,
                            "chapter_index": current_chapter,
                            "chunk_type": "text"
                        })

                    current_chunk = ""
                    current_tokens = 0
                    current_chapter = chapter
                    chunk_start_page = None

                sentence_tokens = count_tokens(sentence)
                # Initialize chunk start page
                if current_chunk == "":
                    chunk_start_page = page_num

                # FIX: Check if adding this sentence would exceed limit
                if current_tokens + sentence_tokens > max_tokens:
                    # Emit current chunk (only if not empty)
                    if current_chunk.strip():
                        chunks.append({
                            "text": current_chunk.strip(),
                            "page_start": chunk_start_page,
                            "page_end": page_num,
                            "chapter_index": current_chapter,
                            "chunk_type": "text"
                        })

                    # Create overlap from previous chunk
                    if current_chunk.strip():
                        overlap_text = enc.decode(
                            enc.encode(current_chunk)[-overlap_tokens:]
                        )
                    else:
                        overlap_text = ""

                    # Start new chunk with overlap + current sentence
                    current_chunk = (overlap_text + " " + sentence).strip() + " "
                    current_tokens = count_tokens(current_chunk)
                    chunk_start_page = page_num
                    
                    # FIX: If even the new chunk with overlap exceeds limit, 
                    # start fresh without overlap
                    if current_tokens > max_tokens:
                        current_chunk = sentence + " "
                        current_tokens = sentence_tokens
                else:
                    # Add sentence to current chunk
                    current_chunk += sentence + " "
                    current_tokens += sentence_tokens
                
    if current_chunk.strip():
        chunks.append({
            "text": current_chunk.strip(),
            "page_start": chunk_start_page,
            "page_end": page_num,
            "chapter_index": current_chapter,
            "chunk_type": "text"
        })

    return chunks

In [4]:
path = r"E:\Semester 5\ML\Machine_Learning_Project\Project.pdf"

chunks= chunk_pdf_for_rag(path)

In [5]:
for chk in chunks[ 50: 55]: 
    print(len(chk['text']))

In [6]:

chunks

[{'text': 'National University of Computer and Emerging Sciences\nIslamabad Campus\nDepartment of AI & DS (fall-2025)\nMachine Learning Lab- Projects\nProject Guidelines\n• Each group must consist of 2 members only. • Read the project description carefully before starting your work. • Make sure every member clearly understands the problem statement, dataset, and\nexpected outcomes. • Clarify any doubts with the instructor before implementation. • The project must be original — plagiarism from classmates, previous projects, or online\nsources will result in zero marks for all group members. • You may refer to tutorials or documentation for learning, but the final implementation\nand report must be your own work. • Only one member per group will upload/submit the project. • The submission should include:\no Code file (.ipynb or .py)\no Project report (PDF format)\no A ReadMe file with:\n▪ Group members’ names and roll numbers\n▪ Short description of the project\n▪ Required libraries/depe

In [ ]:
from langgraph.graph import END, StateGraph, START
from typing import List, Dict, Annotated
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import os
from langchain_community.tools import DuckDuckGoSearchRun


class chatState(Dict):
    messages: Annotated[List[BaseMessage], add_messages]
    llm: None


def get_model():
    llm = ChatGroq(model="moonshotai/kimi-k2-instruct-0905")
    return {'llm': llm}


def chat(chat_state: chatState):
    response = llm.invoke(chat_state["messages"])
    return {"messages": [response]}    

@tool
def search(query: str):
    res = DuckDuckGoSearchRun().invoke(query)
    return res

tools=[search]
tool_node = ToolNode(tools)
load_dotenv()

os.environ['GROQ_API_KEY'] = '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'

llm_with_tools = llm.bind(tools)

graph = StateGraph(chatState)   


graph.add_node("chat", chat)

graph.add_edge(START, "chat")
graph.add_edge("chat", tools_cond)

chatbot = graph.compile()


In [31]:
prompt = """
You are a helpful assistant.
"""
chatbot.invoke({"messages": [HumanMessage(content="who is Quaid e azam?")]})

{'messages': [HumanMessage(content='who is Quaid e azam?', additional_kwargs={}, response_metadata={}, id='58a94c9b-2b46-4f8c-b6b8-04ce0cd421f1'),
  AIMessage(content='Quaid-e-Azam (Urdu for “Great Leader”) is the title given to Muhammad Ali Jinnah, the founder of Pakistan. Born on 25 December 1876 in Karachi, he trained as a barrister in London and became one of India’s most successful lawyers. Jinnah entered politics as a secular nationalist in the Indian National Congress but later joined the All-India Muslim League, arguing that Muslims constituted a separate nation that needed its own state to avoid being dominated by a Hindu-majority independent India. After years of negotiations—especially with the Indian National Congress and the British Raj—he led the demand for a separate homeland for Muslims, culminating in the creation of Pakistan on 14 August 1947. He served as the new nation’s first Governor-General until his death on 11 September 1948.', additional_kwargs={}, response_me

In [27]:
import requests
import os

api_key = os.environ.get("GROQ_API_KEY")
url = "https://api.groq.com/openai/v1/models"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

response.json()

{'object': 'list',
 'data': [{'id': 'openai/gpt-oss-20b',
   'object': 'model',
   'created': 1754407957,
   'owned_by': 'OpenAI',
   'active': True,
   'context_window': 131072,
   'public_apps': None,
   'max_completion_tokens': 65536},
  {'id': 'meta-llama/llama-4-maverick-17b-128e-instruct',
   'object': 'model',
   'created': 1743877158,
   'owned_by': 'Meta',
   'active': True,
   'context_window': 131072,
   'public_apps': None,
   'max_completion_tokens': 8192},
  {'id': 'moonshotai/kimi-k2-instruct-0905',
   'object': 'model',
   'created': 1757046093,
   'owned_by': 'Moonshot AI',
   'active': True,
   'context_window': 262144,
   'public_apps': None,
   'max_completion_tokens': 16384},
  {'id': 'whisper-large-v3',
   'object': 'model',
   'created': 1693721698,
   'owned_by': 'OpenAI',
   'active': True,
   'context_window': 448,
   'public_apps': None,
   'max_completion_tokens': 448},
  {'id': 'openai/gpt-oss-120b',
   'object': 'model',
   'created': 1754408224,
   'owned